# N starvation 

## Which N sources allow MED4 to grow

check out Escher visualization https://escher.readthedocs.io/en/latest/getting_started.html

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import cobra
import numpy as np
import seaborn as sns
import itertools
from matplotlib.colors import LogNorm, Normalize


In [2]:
import subprocess
import time


In [3]:
from cobra.flux_analysis import flux_variability_analysis
from cobra.flux_analysis import production_envelope


In [4]:
from cobra.flux_analysis import (
    single_gene_deletion, single_reaction_deletion, double_gene_deletion,
    double_reaction_deletion)

In [5]:
import signal
from contextlib import contextmanager

@contextmanager
def time_limit(seconds):
    def signal_handler(signum, frame):
        raise TimeoutError
    signal.signal(signal.SIGALRM, signal_handler)
    signal.alarm(seconds)
    try:
        yield
    finally:
        signal.alarm(0)

In [6]:
sns.hls_palette

<function seaborn.palettes.hls_palette(n_colors=6, h=0.01, l=0.6, s=0.65, as_cmap=False)>

In [7]:
# values from Ofaim at el.

PARAMETER_VALUES = [#"Name",     "Reaction ID",          "Lower bound", "UpperBound"
                   ["HCO3",      "HCO3EXcar",            -8,            0],
                   #["Nitrogen",  "AmmoniaEX",            -0.56,         0],
                   #["Phosphate", "FAKEOrthophosphateEX", -0.1,          0],
                   ["Light",     "LightEX",              -150,          0]]
CO2MAX = 0.82

# Block fake reactions
FAKE_TRANSPORT = ["AminosugarsOUT", "FAKEAAOUT", "FAKEABPOUT", "FAKEacpTRANS", "FAKEApoacpTRANS", "FAKEThioredoxinTRANS", 
                  "FreefattyacidsOUT", "7NMeth7carbOUT", "ArtificialproteinOUT", "FADOUT", "LipoylproteinTRANS", "MenaquinoneOUT", 
                  "NicotinateOUT", "THFpolyglutOUT", "Thiamin_dpOUT"]


In [8]:
def get_elements(rid):
    r = model.reactions.get_by_id(rid)
    m = r.reactants[0]    
    elements = m.elements.copy()
    
    res = {
        'reaction' : rid,
        'metabolite' : m.name,
        'N' : 'N' in elements,
        'C' : 'C' in elements,
        
    }
    return res

# Import model and manipulate based on Ofaim at el

In [9]:
import os
model_dpath = os.path.join('..', 'Model_files')
model_fname = 'iSO595v7.xml'
model_fpath = os.path.join(model_dpath, model_fname)


In [10]:
model = cobra.io.read_sbml_model(model_fpath)

Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled


In [11]:
# manipulations copied from Ofaim at el.

remove_blocked = False

# Block H2S
model.reactions.H2SEX.lower_bound = 0

# Block fake transports
for rid in FAKE_TRANSPORT:
    model.reactions.get_by_id(rid).bounds = (0,0)

# Remove blocked reactions
if remove_blocked:
    blocked = cobra.flux_analysis.find_blocked_reactions(model, open_exchanges = True)
    print('blocked', len(blocked), blocked)
    model.remove_reactions([model.reactions.get_by_id(r_id) for r_id in blocked])

# Block maximum CO2 production
model.reactions.CO2EX.bounds = (0, CO2MAX)

for i, row in enumerate(PARAMETER_VALUES):
    # Row: Name, Reaction ID, lower bound, upper bound
    key = row[0]
    reaction_id = row[1]
    lower_bound = row[2]
    upper_bound = row[3]
    r = model.reactions.get_by_id(reaction_id)
    # Fix flux
    r.bounds = (lower_bound, upper_bound)

In [12]:
exchange_ids = [r.id for r in model.exchanges] # + ["R00024"]

In [13]:
print('reactions', len(model.reactions))
print('metabolites', len(model.metabolites))
print('genes', len(model.genes))

reactions 994
metabolites 802
genes 595


In [14]:
model

Name,COBRAModel
Memory address,0x07fa8479c8970
Number of metabolites,802
Number of reactions,994
Number of groups,55
Objective expression,1.0*BIOMASS - 1.0*BIOMASS_reverse_69053
Compartments,"Cytoplasm, Extracellular, unknownCompartment2, Periplasm, unknownCompartment5"


In [15]:
model.reactions.get_by_id('BIOMASS')

Reaction identifier,BIOMASS
Name,Biomass formation
Memory address,0x7fa800fc90a0
Stoichiometry,104.0 ATP[c] + 0.0324116276714271 BioPool[c] + 0.0268408791654006 Bmineral[c] + 0.0559910868023904 Cell_wall[c] + 0.00111921401802897 Free_nucleic_acids[c] + 104.0 H2O[c] + 0.649245416793275... 104.0 ATP + 0.0324116276714271 BioPool + 0.0268408791654006 Trace metals and minerals + 0.0559910868023904 Cell wall Biomass precursor + 0.00111921401802897 Free nucleic acids + 104.0 H2O +...
GPR,
Lower bound,0.0
Upper bound,1000.0


In [16]:
fpath = 'PRO_N_sources.xlsx'
df_growing = pd.read_excel(fpath, sheet_name='uptake')
n_exchange_df = pd.read_excel(fpath, sheet_name='secretion')


# Knockout genes - to find genes required for uptake/secretion

### Partition genes and reactions to essential and non-essential

In [21]:
# open all potential mediums and run gene knockout to see what is blocking growth when all N sources are available 
with model:
    medium = model.medium
    medium["AmmoniaEX"] = 0.0
    for i in df_growing.uptake:
        medium[i] = 1000.0
    model.medium = medium
    model.reactions.BIOMASS.lower_bound = 1e-5
    solution = model.optimize()
    print(model.summary())
    essential_gene_df = single_gene_deletion(model, processes=10)
    print('done genes')
    essential_reaction_df = single_reaction_deletion(model, processes=10)



Objective
1.0 BIOMASS = 1.3183321417861487

Uptake
------
         Metabolite             Reaction      Flux  C-Number C-Flux
         Cadmium[e]            CadmiumEX 6.363E-05         0  0.00%
  Calcium_cation[e]            CalciumEX   0.03529         0  0.00%
    Chloride_ion[e]           ChlorideEX    0.3136         0  0.00%
      Cobalt_ion[e]             CobaltEX  0.002046         0  0.00%
          Copper[e]             CopperEX 0.0001151         0  0.00%
  Orthophosphate[e] FAKEOrthophosphateEX     0.479         0  0.00%
     Glutathione[e]        GlutathioneEX    0.6745         0  0.00%
       Guanosine[e]          GuanosineEX     22.81         0  0.00%
            HCO3[e]            HCO3EXcar         8         1  5.49%
             Fe2[e]               IronEX  0.004261         0  0.00%
       L_Alanine[e]          L_AlanineEX    0.5777         3  1.19%
      L_Arginine[e]         L_ArginineEX    0.5092         6  2.09%
     L_Glutamine[e]        L_GlutamineEX    0.9174        

In [22]:
essential_gene_df.loc[~essential_gene_df.status.isin(['optimal'])]

,ids,growth,status
0,{PMM0572},NaN,infeasible
2,{PMM0287},NaN,infeasible
3,{PMM0494},NaN,infeasible
6,{PMM0243},NaN,infeasible
8,{PMM1582},NaN,infeasible
...,...,...,...
580,{PMM0511},NaN,infeasible
581,{PMM0526},NaN,infeasible
583,{PMM0318},NaN,infeasible
585,{PMM1630},NaN,infeasible


In [23]:
essential_reaction_df.loc[~essential_reaction_df.status.isin(['optimal'])]

,ids,growth,status
18,{R04962},NaN,infeasible
19,{R03182},NaN,infeasible
54,{R04536},NaN,infeasible
68,{R03223},NaN,infeasible
69,{R09959},NaN,infeasible
...,...,...,...
984,{R02719},NaN,infeasible
985,{R05637},NaN,infeasible
986,{tRNAEX},NaN,infeasible
991,{ChlorideEX},NaN,infeasible


In [24]:
non_essential_genes = set().union(*essential_gene_df.loc[essential_gene_df.status.isin(['optimal']), 'ids'].values)

In [25]:
non_essential_reactions = set().union(*essential_reaction_df.loc[essential_reaction_df.status.isin(['optimal']), 'ids'].values)

In [26]:
len(non_essential_genes), len(non_essential_reactions)

(375, 726)

In [32]:
essential_gene_df.to_csv(os.path.join('data', 'MED4_essential_genes.csv'))
essential_reaction_df.to_csv(os.path.join('data', 'MED4_essential_reactions.csv'))


In [35]:
with open(os.path.join('data', 'MED4_del_gene_list.txt'), 'w') as fh:
    fh.writelines(f'{i}\n' for i in non_essential_genes)
with open(os.path.join('data', 'MED4_del_reaction_list.txt'), 'w') as fh:
    fh.writelines(f'{i}\n' for i in non_essential_reactions)


### Try to delete genes and reactions and see what is breaking the model per uptake option

In [57]:
def _uptake_deletion_feeding_with_timeout(dpath, uptake1, deltype, timeout=10):
    try:
        print (uptake1)
        if deltype == 'gene':
            list_file = os.path.join('data', 'MED4_del_gene_list.txt')
        else:
            list_file = os.path.join('data', 'MED4_del_reaction_list.txt')
            
        result = subprocess.run([
            './prochlorococcus_feed_knockout.py', 
            '--uptake', uptake1, 
            '--out_dpath', dpath,
            '--deltype', deltype,
            '--list_file', list_file,
            ],
            capture_output=True, text=True, check=True, timeout=timeout,
        )
        #print("stdout:", result.stdout)
        #print("stderr:", result.stderr)
        return 'success'
        
    except subprocess.CalledProcessError as err:
        print('CalledProcessError', err.returncode)
        print("stdout:", err.stdout)
        print("stderr:", err.stderr)
        return 'error'
                
    except subprocess.TimeoutExpired as err:
        print('TimeoutExpired', err.timeout)
        print("stdout:", err.stdout)
        print("stderr:", err.stderr)
        return 'timeout'



In [58]:
deltype = 'gene'
dpath = os.path.join('data',f'{deltype}_knockout_uptake_20082022')
knockout_success = [
    _uptake_deletion_feeding_with_timeout(dpath, u, deltype, timeout=10)
    for u in df_growing.uptake
]


GuanosineEX B12EX
GuanosineEX BiotinEX
GuanosineEX MethanolEX
GuanosineEX SAdenosyl4methylthio2oxobutanoateEX
TimeoutExpired 10
stdout: b'Scaling...\n A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00\nProblem data seem to be well scaled\nGuanosineEX SAdenosyl4methylthio2oxobutanoateEX Objective\n=========\n1.0 BIOMASS = 0.008999999999998343\n\nUptake\n------\n         Metabolite             Reaction      Flux  C-Number  C-Flux\n         Cadmium[e]            CadmiumEX 4.344E-07         0   0.00%\n  Calcium_cation[e]            CalciumEX 0.0002409         0   0.00%\n    Chloride_ion[e]           ChlorideEX  0.002141         0   0.00%\n      Cobalt_ion[e]             CobaltEX 1.397E-05         0   0.00%\n          Copper[e]             CopperEX  7.86E-07         0   0.00%\n  Orthophosphate[e] FAKEOrthophosphateEX   0.00327         0   0.00%\n       Guanosine[e]          GuanosineEX     30.15         0   0.00%\n            HCO3[e]            HCO3EXcar         8         

GuanosineEX L_CystineEX
GuanosineEX L_GlutamateEX
GuanosineEX L_GlutamineEX
GuanosineEX GlycineEX
GuanosineEX L_HistidineEX
TimeoutExpired 10
stdout: b'Scaling...\n A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00\nProblem data seem to be well scaled\n'
stderr: None
GuanosineEX L_IsoleucineEX
GuanosineEX L_LeucineEX
GuanosineEX L_LysineEX
TimeoutExpired 10
stdout: b'Scaling...\n A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00\nProblem data seem to be well scaled\nGuanosineEX L_LysineEX Objective\n=========\n1.0 BIOMASS = 0.009000023670776042\n\nUptake\n------\n         Metabolite             Reaction      Flux  C-Number  C-Flux\n         Cadmium[e]            CadmiumEX 4.344E-07         0   0.00%\n  Calcium_cation[e]            CalciumEX 0.0002409         0   0.00%\n    Chloride_ion[e]           ChlorideEX  0.002141         0   0.00%\n      Cobalt_ion[e]             CobaltEX 1.397E-05         0   0.00%\n          Copper[e]             CopperEX  7.

GuanosineEX LPSEX
GuanosineEX AcetateEX
GuanosineEX CitrateEX
CalledProcessError 1
stdout: Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled

stderr: Traceback (most recent call last):
  File "/fast_data/Osnat/CC1A3/COMETS/Prochlorococcus_Model/N_Starvation/./prochlorococcus_feed_knockout.py", line 123, in <module>
    resdf = _gene_deletion_feeding(model, args.uptake, args.secretion, args.bound)
  File "/fast_data/Osnat/CC1A3/COMETS/Prochlorococcus_Model/N_Starvation/./prochlorococcus_feed_knockout.py", line 93, in _gene_deletion_feeding
    print(uptake1, sec1, model.summary())
  File "/home/oweissberg/work/tools/anaconda3/lib/python3.9/site-packages/cobra/core/model.py", line 1209, in summary
    return ModelSummary(model=self, solution=solution, fva=fva)
  File "/home/oweissberg/work/tools/anaconda3/lib/python3.9/site-packages/cobra/summary/model_summary.py", line 85, in __init__
    self._generate(model, solution, f

L_AspartateEX UreaEX
L_AspartateEX L_AlanineEX
L_AspartateEX L_ArginineEX
L_AspartateEX L_AsparagineEX
L_AspartateEX L_CystineEX
L_AspartateEX L_GlutamateEX
CalledProcessError 1
stdout: Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled

stderr: Traceback (most recent call last):
  File "/fast_data/Osnat/CC1A3/COMETS/Prochlorococcus_Model/N_Starvation/./prochlorococcus_feed_knockout.py", line 123, in <module>
    resdf = _gene_deletion_feeding(model, args.uptake, args.secretion, args.bound)
  File "/fast_data/Osnat/CC1A3/COMETS/Prochlorococcus_Model/N_Starvation/./prochlorococcus_feed_knockout.py", line 93, in _gene_deletion_feeding
    print(uptake1, sec1, model.summary())
  File "/home/oweissberg/work/tools/anaconda3/lib/python3.9/site-packages/cobra/core/model.py", line 1209, in summary
    return ModelSummary(model=self, solution=solution, fva=fva)
  File "/home/oweissberg/work/tools/anaconda3/lib/python3.9/site-packa

L_AspartateEX SuccinateEX
L_AspartateEX ThymidineEX
TimeoutExpired 10
stdout: b'Scaling...\n A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00\nProblem data seem to be well scaled\nL_AspartateEX ThymidineEX Objective\n=========\n1.0 BIOMASS = 0.008999999999997562\n\nUptake\n------\n         Metabolite             Reaction      Flux  C-Number C-Flux\n         Cadmium[e]            CadmiumEX 4.344E-07         0  0.00%\n  Calcium_cation[e]            CalciumEX 0.0002409         0  0.00%\n    Chloride_ion[e]           ChlorideEX  0.002141         0  0.00%\n      Cobalt_ion[e]             CobaltEX 1.397E-05         0  0.00%\n          Copper[e]             CopperEX  7.86E-07         0  0.00%\n  Orthophosphate[e] FAKEOrthophosphateEX   0.00327         0  0.00%\n             H2O[e]                H2OEX     22.46         0  0.00%\n            HCO3[e]            HCO3EXcar     1.924         1  2.54%\n             Fe2[e]               IronEX 2.909E-05         0  0.00%\n     L_As

...

TimeoutExpired 10
stdout: b'Scaling...\n A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00\nProblem data seem to be well scaled\n'
stderr: None
L_ValineEX B12EX
TimeoutExpired 10
stdout: b'Scaling...\n A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00\nProblem data seem to be well scaled\n'
stderr: None
L_ValineEX BiotinEX
TimeoutExpired 10
stdout: b'Scaling...\n A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00\nProblem data seem to be well scaled\n'
stderr: None
L_ValineEX MethanolEX
TimeoutExpired 10
stdout: b'Scaling...\n A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00\nProblem data seem to be well scaled\n'
stderr: None
L_ValineEX SAdenosyl4methylthio2oxobutanoateEX
TimeoutExpired 10
stdout: b'Scaling...\n A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00\nProblem data seem to be well scaled\n'
stderr: None
L_ValineEX SMethyl5thioDriboseEX
TimeoutExpired 10
stdout: b'Scaling...\n A: min|aij

TimeoutExpired 10
stdout: b'Scaling...\n A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00\nProblem data seem to be well scaled\n'
stderr: None
L_ValineEX L_TryptophanEX
TimeoutExpired 10
stdout: b'Scaling...\n A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00\nProblem data seem to be well scaled\n'
stderr: None
L_ValineEX L_TyrosineEX
TimeoutExpired 10
stdout: b'Scaling...\n A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00\nProblem data seem to be well scaled\n'
stderr: None
L_ValineEX PhospholipidEX
TimeoutExpired 10
stdout: b'Scaling...\n A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00\nProblem data seem to be well scaled\n'
stderr: None
L_ValineEX LPSEX
TimeoutExpired 10
stdout: b'Scaling...\n A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00\nProblem data seem to be well scaled\n'
stderr: None
L_ValineEX AcetateEX
TimeoutExpired 10
stdout: b'Scaling...\n A: min|aij| =  1.000e+00  max|aij| 

TimeoutExpired 10
stdout: b'Scaling...\n A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00\nProblem data seem to be well scaled\n'
stderr: None
GuanineEX ThiosulfateEX
TimeoutExpired 10
stdout: b'Scaling...\n A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00\nProblem data seem to be well scaled\n'
stderr: None
GuanineEX UreaEX
TimeoutExpired 10
stdout: b'Scaling...\n A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00\nProblem data seem to be well scaled\n'
stderr: None
GuanineEX L_AlanineEX
TimeoutExpired 10
stdout: b'Scaling...\n A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00\nProblem data seem to be well scaled\n'
stderr: None
GuanineEX L_ArginineEX
TimeoutExpired 10
stdout: b'Scaling...\n A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00\nProblem data seem to be well scaled\n'
stderr: None
GuanineEX L_AsparagineEX
TimeoutExpired 10
stdout: b'Scaling...\n A: min|aij| =  1.000e+00  max|aij| =  

TimeoutExpired 10
stdout: b'Scaling...\n A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00\nProblem data seem to be well scaled\n'
stderr: None
GuanineEX FumarateEX
TimeoutExpired 10
stdout: b'Scaling...\n A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00\nProblem data seem to be well scaled\n'
stderr: None
GuanineEX GlutathioneEX
TimeoutExpired 10
stdout: b'Scaling...\n A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00\nProblem data seem to be well scaled\n'
stderr: None
GuanineEX 4_Methyl_2_oxopentanoateEX
TimeoutExpired 10
stdout: b'Scaling...\n A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00\nProblem data seem to be well scaled\n'
stderr: None
GuanineEX PantothenateEX
TimeoutExpired 10
stdout: b'Scaling...\n A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00\nProblem data seem to be well scaled\nGuanineEX PantothenateEX Objective\n=========\n1.0 BIOMASS = 0.009000068603555504\n\nUptake\n-----

TimeoutExpired 10
stdout: b'Scaling...\n A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00\nProblem data seem to be well scaled\n'
stderr: None
GuanineEX EthanolEX
TimeoutExpired 10
stdout: b'Scaling...\n A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00\nProblem data seem to be well scaled\n'
stderr: None
GuanineEX S_MalateEX
TimeoutExpired 10
stdout: b'Scaling...\n A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00\nProblem data seem to be well scaled\n'
stderr: None
GuanineEX H2O2EX
TimeoutExpired 10
stdout: b'Scaling...\n A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00\nProblem data seem to be well scaled\n'
stderr: None


In [28]:
def _gene_deletion_uptake(uptake1):
    with model:
        medium = model.medium
        medium["AmmoniaEX"] = 0.0
        medium[uptake1] = 1000.0
        model.medium = medium
        model.reactions.get_by_id(uptake1).upper_bound = -1e-5
        #model.reactions.get_by_id(sec1).upper_bound = maxflux
        #model.reactions.get_by_id(sec1).lower_bound = maxflux
        model.reactions.BIOMASS.lower_bound = 1e-5

        #model.reactions.BIOMASS.upper_bound = 1e-2
        #model
        solution = model.optimize()
        print(uptake1, model.summary())
        try:
            with time_limit(1):
                df = single_gene_deletion(model, processes=10, gene_list=list(non_essential_genes))
                df['uptake'] = uptake1
                #df['secretion'] = sec1
                df = df.loc[df.status != 'optimal']
                return df
        except TimeoutError as e:
            return pd.DataFrame.from_records([dict(uptake=uptake1, status='timeout')])            
            print(uptake1, "Timed out!")


In [ ]:
def _reaction_deletion_uptake(uptake1):
    with model:
        medium = model.medium
        medium["AmmoniaEX"] = 0.0
        medium[uptake1] = 1000.0
        model.medium = medium
        model.reactions.get_by_id(uptake1).upper_bound = -1e-5
        #model.reactions.get_by_id(sec1).upper_bound = maxflux
        #model.reactions.get_by_id(sec1).lower_bound = maxflux
        model.reactions.BIOMASS.lower_bound = 1e-5

        #model.reactions.BIOMASS.upper_bound = 1e-2
        #model
        solution = model.optimize()
        print(uptake1, model.summary())
        df = single_reaction_deletion(model, processes=10, gene_list=list(non_essential_reactions))
        df['uptake'] = uptake1
        #df['secretion'] = sec1
        df = df.loc[df.status != 'optimal']
        return df


In [ ]:
uptake_gene_knockout_df = pd.concat([_gene_deletion_uptake(u) for u in df_growing['uptake'] ])


GuanosineEX Objective
1.0 BIOMASS = 1.239447402033875

Uptake
------
         Metabolite             Reaction      Flux  C-Number  C-Flux
         Cadmium[e]            CadmiumEX 5.982E-05         0   0.00%
  Calcium_cation[e]            CalciumEX   0.03317         0   0.00%
    Chloride_ion[e]           ChlorideEX    0.2949         0   0.00%
      Cobalt_ion[e]             CobaltEX  0.001923         0   0.00%
          Copper[e]             CopperEX 0.0001083         0   0.00%
  Orthophosphate[e] FAKEOrthophosphateEX    0.4503         0   0.00%
       Guanosine[e]          GuanosineEX     48.05         0   0.00%
            HCO3[e]            HCO3EXcar         8         1 100.00%
             Fe2[e]               IronEX  0.004006         0   0.00%
          Photon[e]              LightEX    0.2252         0   0.00%
Magnesium_cation[e]          MagnesiumEX    0.1969         0   0.00%
      Molybdenum[e]         MolybdenumEX 8.546E-06         0   0.00%
          Oxygen[e]               

In [ ]:
uptake_reaction_knockout_df = pd.concat([_reaction_deletion_uptake(u) for u in df_growing['uptake'] ])


In [35]:
n_exchange_df.head()                          

,secretion,minimum,maximum,uptake
8,B12EX,0.000000,7.941910,GuanosineEX
9,BiotinEX,0.000000,1.985477,GuanosineEX
14,MethanolEX,0.000042,1.985519,GuanosineEX
16,SAdenosyl4methylthio2oxobutanoateEX,0.000042,1.985519,GuanosineEX
17,SMethyl5thioDriboseEX,0.000000,15.884229,GuanosineEX


In [54]:
n_exchange_df.shape

(881, 4)

In [58]:
tdf = n_exchange_df
dpath = 'gene_knockout_secreted_18082022'
knockout_success = [
    _gene_deletion_feeding_with_timeout(dpath, u, s, m, timeout=10)
    for (u, s, m) in zip(tdf['uptake'], tdf['secretion'], tdf['maximum'])
]


GuanosineEX B12EX
GuanosineEX BiotinEX
GuanosineEX MethanolEX
GuanosineEX SAdenosyl4methylthio2oxobutanoateEX
TimeoutExpired 10
stdout: b'Scaling...\n A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00\nProblem data seem to be well scaled\nGuanosineEX SAdenosyl4methylthio2oxobutanoateEX Objective\n=========\n1.0 BIOMASS = 0.008999999999998343\n\nUptake\n------\n         Metabolite             Reaction      Flux  C-Number  C-Flux\n         Cadmium[e]            CadmiumEX 4.344E-07         0   0.00%\n  Calcium_cation[e]            CalciumEX 0.0002409         0   0.00%\n    Chloride_ion[e]           ChlorideEX  0.002141         0   0.00%\n      Cobalt_ion[e]             CobaltEX 1.397E-05         0   0.00%\n          Copper[e]             CopperEX  7.86E-07         0   0.00%\n  Orthophosphate[e] FAKEOrthophosphateEX   0.00327         0   0.00%\n       Guanosine[e]          GuanosineEX     30.15         0   0.00%\n            HCO3[e]            HCO3EXcar         8         

GuanosineEX L_CystineEX
GuanosineEX L_GlutamateEX
GuanosineEX L_GlutamineEX
GuanosineEX GlycineEX
GuanosineEX L_HistidineEX
TimeoutExpired 10
stdout: b'Scaling...\n A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00\nProblem data seem to be well scaled\n'
stderr: None
GuanosineEX L_IsoleucineEX
GuanosineEX L_LeucineEX
GuanosineEX L_LysineEX
TimeoutExpired 10
stdout: b'Scaling...\n A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00\nProblem data seem to be well scaled\nGuanosineEX L_LysineEX Objective\n=========\n1.0 BIOMASS = 0.009000023670776042\n\nUptake\n------\n         Metabolite             Reaction      Flux  C-Number  C-Flux\n         Cadmium[e]            CadmiumEX 4.344E-07         0   0.00%\n  Calcium_cation[e]            CalciumEX 0.0002409         0   0.00%\n    Chloride_ion[e]           ChlorideEX  0.002141         0   0.00%\n      Cobalt_ion[e]             CobaltEX 1.397E-05         0   0.00%\n          Copper[e]             CopperEX  7.

GuanosineEX LPSEX
GuanosineEX AcetateEX
GuanosineEX CitrateEX
CalledProcessError 1
stdout: Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled

stderr: Traceback (most recent call last):
  File "/fast_data/Osnat/CC1A3/COMETS/Prochlorococcus_Model/N_Starvation/./prochlorococcus_feed_knockout.py", line 123, in <module>
    resdf = _gene_deletion_feeding(model, args.uptake, args.secretion, args.bound)
  File "/fast_data/Osnat/CC1A3/COMETS/Prochlorococcus_Model/N_Starvation/./prochlorococcus_feed_knockout.py", line 93, in _gene_deletion_feeding
    print(uptake1, sec1, model.summary())
  File "/home/oweissberg/work/tools/anaconda3/lib/python3.9/site-packages/cobra/core/model.py", line 1209, in summary
    return ModelSummary(model=self, solution=solution, fva=fva)
  File "/home/oweissberg/work/tools/anaconda3/lib/python3.9/site-packages/cobra/summary/model_summary.py", line 85, in __init__
    self._generate(model, solution, f

L_AspartateEX UreaEX
L_AspartateEX L_AlanineEX
L_AspartateEX L_ArginineEX
L_AspartateEX L_AsparagineEX
L_AspartateEX L_CystineEX
L_AspartateEX L_GlutamateEX
CalledProcessError 1
stdout: Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled

stderr: Traceback (most recent call last):
  File "/fast_data/Osnat/CC1A3/COMETS/Prochlorococcus_Model/N_Starvation/./prochlorococcus_feed_knockout.py", line 123, in <module>
    resdf = _gene_deletion_feeding(model, args.uptake, args.secretion, args.bound)
  File "/fast_data/Osnat/CC1A3/COMETS/Prochlorococcus_Model/N_Starvation/./prochlorococcus_feed_knockout.py", line 93, in _gene_deletion_feeding
    print(uptake1, sec1, model.summary())
  File "/home/oweissberg/work/tools/anaconda3/lib/python3.9/site-packages/cobra/core/model.py", line 1209, in summary
    return ModelSummary(model=self, solution=solution, fva=fva)
  File "/home/oweissberg/work/tools/anaconda3/lib/python3.9/site-packa

L_AspartateEX SuccinateEX
L_AspartateEX ThymidineEX
TimeoutExpired 10
stdout: b'Scaling...\n A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00\nProblem data seem to be well scaled\nL_AspartateEX ThymidineEX Objective\n=========\n1.0 BIOMASS = 0.008999999999997562\n\nUptake\n------\n         Metabolite             Reaction      Flux  C-Number C-Flux\n         Cadmium[e]            CadmiumEX 4.344E-07         0  0.00%\n  Calcium_cation[e]            CalciumEX 0.0002409         0  0.00%\n    Chloride_ion[e]           ChlorideEX  0.002141         0  0.00%\n      Cobalt_ion[e]             CobaltEX 1.397E-05         0  0.00%\n          Copper[e]             CopperEX  7.86E-07         0  0.00%\n  Orthophosphate[e] FAKEOrthophosphateEX   0.00327         0  0.00%\n             H2O[e]                H2OEX     22.46         0  0.00%\n            HCO3[e]            HCO3EXcar     1.924         1  2.54%\n             Fe2[e]               IronEX 2.909E-05         0  0.00%\n     L_As

...

TimeoutExpired 10
stdout: b'Scaling...\n A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00\nProblem data seem to be well scaled\n'
stderr: None
L_ValineEX B12EX
TimeoutExpired 10
stdout: b'Scaling...\n A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00\nProblem data seem to be well scaled\n'
stderr: None
L_ValineEX BiotinEX
TimeoutExpired 10
stdout: b'Scaling...\n A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00\nProblem data seem to be well scaled\n'
stderr: None
L_ValineEX MethanolEX
TimeoutExpired 10
stdout: b'Scaling...\n A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00\nProblem data seem to be well scaled\n'
stderr: None
L_ValineEX SAdenosyl4methylthio2oxobutanoateEX
TimeoutExpired 10
stdout: b'Scaling...\n A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00\nProblem data seem to be well scaled\n'
stderr: None
L_ValineEX SMethyl5thioDriboseEX
TimeoutExpired 10
stdout: b'Scaling...\n A: min|aij

TimeoutExpired 10
stdout: b'Scaling...\n A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00\nProblem data seem to be well scaled\n'
stderr: None
L_ValineEX L_TryptophanEX
TimeoutExpired 10
stdout: b'Scaling...\n A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00\nProblem data seem to be well scaled\n'
stderr: None
L_ValineEX L_TyrosineEX
TimeoutExpired 10
stdout: b'Scaling...\n A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00\nProblem data seem to be well scaled\n'
stderr: None
L_ValineEX PhospholipidEX
TimeoutExpired 10
stdout: b'Scaling...\n A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00\nProblem data seem to be well scaled\n'
stderr: None
L_ValineEX LPSEX
TimeoutExpired 10
stdout: b'Scaling...\n A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00\nProblem data seem to be well scaled\n'
stderr: None
L_ValineEX AcetateEX
TimeoutExpired 10
stdout: b'Scaling...\n A: min|aij| =  1.000e+00  max|aij| 

TimeoutExpired 10
stdout: b'Scaling...\n A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00\nProblem data seem to be well scaled\n'
stderr: None
GuanineEX ThiosulfateEX
TimeoutExpired 10
stdout: b'Scaling...\n A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00\nProblem data seem to be well scaled\n'
stderr: None
GuanineEX UreaEX
TimeoutExpired 10
stdout: b'Scaling...\n A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00\nProblem data seem to be well scaled\n'
stderr: None
GuanineEX L_AlanineEX
TimeoutExpired 10
stdout: b'Scaling...\n A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00\nProblem data seem to be well scaled\n'
stderr: None
GuanineEX L_ArginineEX
TimeoutExpired 10
stdout: b'Scaling...\n A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00\nProblem data seem to be well scaled\n'
stderr: None
GuanineEX L_AsparagineEX
TimeoutExpired 10
stdout: b'Scaling...\n A: min|aij| =  1.000e+00  max|aij| =  

TimeoutExpired 10
stdout: b'Scaling...\n A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00\nProblem data seem to be well scaled\n'
stderr: None
GuanineEX FumarateEX
TimeoutExpired 10
stdout: b'Scaling...\n A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00\nProblem data seem to be well scaled\n'
stderr: None
GuanineEX GlutathioneEX
TimeoutExpired 10
stdout: b'Scaling...\n A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00\nProblem data seem to be well scaled\n'
stderr: None
GuanineEX 4_Methyl_2_oxopentanoateEX
TimeoutExpired 10
stdout: b'Scaling...\n A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00\nProblem data seem to be well scaled\n'
stderr: None
GuanineEX PantothenateEX
TimeoutExpired 10
stdout: b'Scaling...\n A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00\nProblem data seem to be well scaled\nGuanineEX PantothenateEX Objective\n=========\n1.0 BIOMASS = 0.009000068603555504\n\nUptake\n-----

TimeoutExpired 10
stdout: b'Scaling...\n A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00\nProblem data seem to be well scaled\n'
stderr: None
GuanineEX EthanolEX
TimeoutExpired 10
stdout: b'Scaling...\n A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00\nProblem data seem to be well scaled\n'
stderr: None
GuanineEX S_MalateEX
TimeoutExpired 10
stdout: b'Scaling...\n A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00\nProblem data seem to be well scaled\n'
stderr: None
GuanineEX H2O2EX
TimeoutExpired 10
stdout: b'Scaling...\n A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00\nProblem data seem to be well scaled\n'
stderr: None


In [62]:
n_exchange_df['knockout_success1'] = knockout_success

/tmp/ipykernel_1752018/3246280673.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  n_exchange_df['knockout_success1'] = knockout_success


In [64]:
n_exchange_df.loc[n_exchange_df.knockout_success1 != 'success']

,secretion,minimum,maximum,uptake,knockout_success1
16,SAdenosyl4methylthio2oxobutanoateEX,0.000042,1.985519,GuanosineEX,timeout
37,H2SEX,0.000000,717.880829,GuanosineEX,timeout
56,L_AsparagineEX,0.000000,334.855695,GuanosineEX,error
62,L_HistidineEX,0.000000,222.465588,GuanosineEX,timeout
65,L_LysineEX,0.000000,268.071714,GuanosineEX,timeout
...,...,...,...,...,...
1610,PutrescineEX,0.000000,0.259757,GuanineEX,timeout
1611,SpermidineEX,0.000000,0.134414,GuanineEX,timeout
1613,EthanolEX,0.000000,0.389983,GuanineEX,timeout
1614,S_MalateEX,0.000000,1.298786,GuanineEX,timeout


In [66]:
n_exchange_df.loc[n_exchange_df.knockout_success1 == 'success']

,secretion,minimum,maximum,uptake,knockout_success1
8,B12EX,0.000000,7.941910,GuanosineEX,success
9,BiotinEX,0.000000,1.985477,GuanosineEX,success
14,MethanolEX,0.000042,1.985519,GuanosineEX,success
17,SMethyl5thioDriboseEX,0.000000,15.884229,GuanosineEX,success
22,GlycolateEX,0.000000,1000.000000,GuanosineEX,success
...,...,...,...,...,...
210,B12EX,0.000000,0.185132,GlutathioneEX,success
211,BiotinEX,0.000000,0.269283,GlutathioneEX,success
216,MethanolEX,0.000042,0.269325,GlutathioneEX,success
218,SAdenosyl4methylthio2oxobutanoateEX,0.000042,0.269325,GlutathioneEX,success


In [67]:
n_exchange_df.knockout_success1.value_counts()

timeout    776
success     94
error       11
Name: knockout_success1, dtype: int64

In [ ]:
tdf = n_exchange_df.loc[n_exchange_df.knockout_success1 == 'timeout']
dpath = 'gene_knockout_secreted_18082022'
knockout_success = [
    _gene_deletion_feeding_with_timeout(dpath, u, s, m, timeout=20)
    for (u, s, m) in zip(tdf['uptake'], tdf['secretion'], tdf['maximum'])
]


GuanosineEX SAdenosyl4methylthio2oxobutanoateEX
TimeoutExpired 20
stdout: b'Scaling...\n A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00\nProblem data seem to be well scaled\nGuanosineEX SAdenosyl4methylthio2oxobutanoateEX Objective\n=========\n1.0 BIOMASS = 0.008999999999998343\n\nUptake\n------\n         Metabolite             Reaction      Flux  C-Number  C-Flux\n         Cadmium[e]            CadmiumEX 4.344E-07         0   0.00%\n  Calcium_cation[e]            CalciumEX 0.0002409         0   0.00%\n    Chloride_ion[e]           ChlorideEX  0.002141         0   0.00%\n      Cobalt_ion[e]             CobaltEX 1.397E-05         0   0.00%\n          Copper[e]             CopperEX  7.86E-07         0   0.00%\n  Orthophosphate[e] FAKEOrthophosphateEX   0.00327         0   0.00%\n       Guanosine[e]          GuanosineEX     30.15         0   0.00%\n            HCO3[e]            HCO3EXcar         8         1 100.00%\n               H[e]                  HEX    0.9863 

GlutathioneEX L_LysineEX
GlutathioneEX L_MethionineEX
GlutathioneEX L_PhenylalanineEX
GlutathioneEX L_ProlineEX
GlutathioneEX L_SerineEX
GlutathioneEX L_ThreonineEX
GlutathioneEX L_TryptophanEX
GlutathioneEX L_TyrosineEX
GlutathioneEX L_ValineEX
GlutathioneEX PhospholipidEX
GlutathioneEX LPSEX
GlutathioneEX AcetateEX
GlutathioneEX CitrateEX
GlutathioneEX FormateEX
GlutathioneEX PyruvateEX
GlutathioneEX FumarateEX
GlutathioneEX 4_Methyl_2_oxopentanoateEX
GlutathioneEX PantothenateEX
GlutathioneEX SuccinateEX
GlutathioneEX ThymidineEX
GlutathioneEX XanthosineEX
GlutathioneEX 4_AminobenzoateEX
GlutathioneEX 5_MethylthioadenosineEX
GlutathioneEX AdenineEX
GlutathioneEX DethiobiotinEX
TimeoutExpired 20
stdout: b'Scaling...\n A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00\nProblem data seem to be well scaled\nGlutathioneEX DethiobiotinEX Objective\n=========\n1.0 BIOMASS = 0.009000019890323584\n\nUptake\n------\n         Metabolite             Reaction      Flux  C-Numbe

L_GlutamineEX ThiosulfateEX
L_GlutamineEX UreaEX
L_GlutamineEX L_AlanineEX
L_GlutamineEX L_ArginineEX
L_GlutamineEX L_AsparagineEX
L_GlutamineEX L_AspartateEX
L_GlutamineEX L_CystineEX
L_GlutamineEX L_GlutamateEX
L_GlutamineEX GlycineEX
TimeoutExpired 20
stdout: b'Scaling...\n A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00\nProblem data seem to be well scaled\nL_GlutamineEX GlycineEX Objective\n=========\n1.0 BIOMASS = 0.009000070295363018\n\nUptake\n------\n         Metabolite             Reaction      Flux  C-Number C-Flux\n         Cadmium[e]            CadmiumEX 4.344E-07         0  0.00%\n  Calcium_cation[e]            CalciumEX 0.0002409         0  0.00%\n    Chloride_ion[e]           ChlorideEX  0.002141         0  0.00%\n      Cobalt_ion[e]             CobaltEX 1.397E-05         0  0.00%\n          Copper[e]             CopperEX  7.86E-07         0  0.00%\n  Orthophosphate[e] FAKEOrthophosphateEX   0.00327         0  0.00%\n            HCO3[e]            HC

L_GlutamineEX 4_Methyl_2_oxopentanoateEX
L_GlutamineEX PantothenateEX
L_GlutamineEX SuccinateEX
L_GlutamineEX ThymidineEX
L_GlutamineEX XanthosineEX
L_GlutamineEX 4_AminobenzoateEX
L_GlutamineEX 5_MethylthioadenosineEX
L_GlutamineEX AdenineEX
TimeoutExpired 20
stdout: b'Scaling...\n A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00\nProblem data seem to be well scaled\nL_GlutamineEX AdenineEX Objective\n=========\n1.0 BIOMASS = 0.009000039727490833\n\nUptake\n------\n         Metabolite             Reaction      Flux  C-Number C-Flux\n         Cadmium[e]            CadmiumEX 4.344E-07         0  0.00%\n  Calcium_cation[e]            CalciumEX 0.0002409         0  0.00%\n    Chloride_ion[e]           ChlorideEX  0.002141         0  0.00%\n      Cobalt_ion[e]             CobaltEX 1.397E-05         0  0.00%\n          Copper[e]             CopperEX  7.86E-07         0  0.00%\n  Orthophosphate[e] FAKEOrthophosphateEX   0.00327         0  0.00%\n            HCO3[e]        

L_GlutamineEX SpermidineEX
L_GlutamineEX EthanolEX
L_GlutamineEX S_MalateEX
L_GlutamineEX H2O2EX
CalledProcessError 1
stdout: Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled

stderr: Traceback (most recent call last):
  File "/fast_data/Osnat/CC1A3/COMETS/Prochlorococcus_Model/N_Starvation/./prochlorococcus_feed_knockout.py", line 123, in <module>
    resdf = _gene_deletion_feeding(model, args.uptake, args.secretion, args.bound)
  File "/fast_data/Osnat/CC1A3/COMETS/Prochlorococcus_Model/N_Starvation/./prochlorococcus_feed_knockout.py", line 93, in _gene_deletion_feeding
    print(uptake1, sec1, model.summary())
  File "/home/oweissberg/work/tools/anaconda3/lib/python3.9/site-packages/cobra/core/model.py", line 1209, in summary
    return ModelSummary(model=self, solution=solution, fva=fva)
  File "/home/oweissberg/work/tools/anaconda3/lib/python3.9/site-packages/cobra/summary/model_summary.py", line 85, in __init__
  

...

TimeoutExpired 20
stdout: b'Scaling...\n A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00\nProblem data seem to be well scaled\nL_ValineEX L_ThreonineEX Objective\n=========\n1.0 BIOMASS = 0.00900005574726269\n\nUptake\n------\n         Metabolite             Reaction      Flux  C-Number C-Flux\n         Cadmium[e]            CadmiumEX 4.344E-07         0  0.00%\n  Calcium_cation[e]            CalciumEX 0.0002409         0  0.00%\n    Chloride_ion[e]           ChlorideEX  0.002141         0  0.00%\n      Cobalt_ion[e]             CobaltEX 1.397E-05         0  0.00%\n          Copper[e]             CopperEX  7.86E-07         0  0.00%\n  Orthophosphate[e] FAKEOrthophosphateEX   0.00327         0  0.00%\n            HCO3[e]            HCO3EXcar     4.759         1 51.14%\n             Fe2[e]               IronEX 2.909E-05         0  0.00%\n        L_Valine[e]           L_ValineEX    0.9094         5 48.86%\n          Photon[e]              LightEX     51.41         0  0

TimeoutExpired 20
stdout: b'Scaling...\n A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00\nProblem data seem to be well scaled\nL_ValineEX PhospholipidEX Objective\n=========\n1.0 BIOMASS = 0.009000060815229027\n\nUptake\n------\n         Metabolite             Reaction      Flux  C-Number C-Flux\n         Cadmium[e]            CadmiumEX 4.344E-07         0  0.00%\n  Calcium_cation[e]            CalciumEX 0.0002409         0  0.00%\n    Chloride_ion[e]           ChlorideEX  0.002141         0  0.00%\n      Cobalt_ion[e]             CobaltEX 1.397E-05         0  0.00%\n          Copper[e]             CopperEX  7.86E-07         0  0.00%\n  Orthophosphate[e] FAKEOrthophosphateEX     1.389         0  0.00%\n            HCO3[e]            HCO3EXcar     4.759         1 92.41%\n             Fe2[e]               IronEX 2.909E-05         0  0.00%\n        L_Valine[e]           L_ValineEX   0.07817         5  7.59%\n          Photon[e]              LightEX     50.17         0 

TimeoutExpired 20
stdout: b'Scaling...\n A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00\nProblem data seem to be well scaled\nL_ValineEX CitrateEX Objective\n=========\n1.0 BIOMASS = 0.00900002533957864\n\nUptake\n------\n         Metabolite             Reaction      Flux  C-Number C-Flux\n         Cadmium[e]            CadmiumEX 4.344E-07         0  0.00%\n  Calcium_cation[e]            CalciumEX 0.0002409         0  0.00%\n    Chloride_ion[e]           ChlorideEX  0.002141         0  0.00%\n      Cobalt_ion[e]             CobaltEX 1.397E-05         0  0.00%\n          Copper[e]             CopperEX  7.86E-07         0  0.00%\n  Orthophosphate[e] FAKEOrthophosphateEX   0.00327         0  0.00%\n            HCO3[e]            HCO3EXcar     5.481         1 93.34%\n             Fe2[e]               IronEX 2.909E-05         0  0.00%\n        L_Valine[e]           L_ValineEX   0.07817         5  6.66%\n          Photon[e]              LightEX     47.06         0  0.00%

TimeoutExpired 20
stdout: b'Scaling...\n A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00\nProblem data seem to be well scaled\nL_ValineEX FumarateEX Objective\n=========\n1.0 BIOMASS = 0.009000055747262632\n\nUptake\n------\n         Metabolite             Reaction      Flux  C-Number C-Flux\n         Cadmium[e]            CadmiumEX 4.344E-07         0  0.00%\n  Calcium_cation[e]            CalciumEX 0.0002409         0  0.00%\n    Chloride_ion[e]           ChlorideEX  0.002141         0  0.00%\n      Cobalt_ion[e]             CobaltEX 1.397E-05         0  0.00%\n          Copper[e]             CopperEX  7.86E-07         0  0.00%\n  Orthophosphate[e] FAKEOrthophosphateEX   0.00327         0  0.00%\n            HCO3[e]            HCO3EXcar     6.035         1 93.92%\n             Fe2[e]               IronEX 2.909E-05         0  0.00%\n        L_Valine[e]           L_ValineEX   0.07817         5  6.08%\n          Photon[e]              LightEX     48.46         0  0.0

L_ValineEX ThymidineEX
L_ValineEX XanthosineEX
L_ValineEX 4_AminobenzoateEX
L_ValineEX 5_MethylthioadenosineEX
L_ValineEX AdenineEX
L_ValineEX DethiobiotinEX
L_ValineEX GuanineEX
TimeoutExpired 20
stdout: b'Scaling...\n A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00\nProblem data seem to be well scaled\nL_ValineEX GuanineEX Objective\n=========\n1.0 BIOMASS = 0.009000048871926507\n\nUptake\n------\n         Metabolite             Reaction      Flux  C-Number C-Flux\n         Cadmium[e]            CadmiumEX 4.344E-07         0  0.00%\n  Calcium_cation[e]            CalciumEX 0.0002409         0  0.00%\n    Chloride_ion[e]           ChlorideEX  0.002141         0  0.00%\n      Cobalt_ion[e]             CobaltEX 1.397E-05         0  0.00%\n          Copper[e]             CopperEX  7.86E-07         0  0.00%\n  Orthophosphate[e] FAKEOrthophosphateEX   0.00327         0  0.00%\n            HCO3[e]            HCO3EXcar     4.977         1 45.93%\n             Fe2[e]      

In [73]:
len(knockout_success)

n_exchange_df.loc[n_exchange_df.knockout_success1 == 'timeout', 'knockout_success2'] = knockout_success

/home/oweissberg/work/tools/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
/home/oweissberg/work/tools/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [74]:
n_exchange_df.loc[
    n_exchange_df.knockout_success1 != 'timeout', 'knockout_success2'
] = n_exchange_df.loc[
    n_exchange_df.knockout_success1 != 'timeout', 'knockout_success1'
]

/home/oweissberg/work/tools/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [76]:
n_exchange_df.knockout_success2.value_counts()

success    610
error      180
timeout     91
Name: knockout_success2, dtype: int64

In [77]:
tdf = n_exchange_df.loc[n_exchange_df.knockout_success2 == 'timeout']
dpath = 'gene_knockout_secreted_18082022'
knockout_success = [
    _gene_deletion_feeding_with_timeout(dpath, u, s, m, timeout=20)
    for (u, s, m) in zip(tdf['uptake'], tdf['secretion'], tdf['maximum'])
]


GuanosineEX SAdenosyl4methylthio2oxobutanoateEX
GuanosineEX L_HistidineEX
TimeoutExpired 20
stdout: b'Scaling...\n A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00\nProblem data seem to be well scaled\n'
stderr: None
L_AspartateEX 5_MethylthioadenosineEX
GlutathioneEX L_IsoleucineEX
GlutathioneEX DethiobiotinEX
GlutathioneEX GuanosineEX
L_GlutamineEX GlycineEX
L_GlutamineEX PhospholipidEX
L_GlutamineEX FumarateEX
L_GlutamineEX AdenineEX
L_GlutamineEX DethiobiotinEX
L_GlutamineEX GuanosineEX
TimeoutExpired 20
stdout: b'Scaling...\n A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00\nProblem data seem to be well scaled\nL_GlutamineEX GuanosineEX Objective\n=========\n1.0 BIOMASS = 0.009000039727491204\n\nUptake\n------\n         Metabolite             Reaction      Flux  C-Number C-Flux\n         Cadmium[e]            CadmiumEX 4.344E-07         0  0.00%\n  Calcium_cation[e]            CalciumEX 0.0002409         0  0.00%\n    Chloride_ion[e]          

L_LeucineEX L_MethionineEX
TimeoutExpired 20
stdout: b'Scaling...\n A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00\nProblem data seem to be well scaled\n'
stderr: None
GlycineEX SMethyl5thioDriboseEX
TimeoutExpired 20
stdout: b'Scaling...\n A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00\nProblem data seem to be well scaled\n'
stderr: None
GlycineEX GlycogenEX
GlycineEX L_AsparagineEX
GlycineEX SuccinateEX
GlycineEX 4_AminobenzoateEX
TimeoutExpired 20
stdout: b'Scaling...\n A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00\nProblem data seem to be well scaled\nGlycineEX 4_AminobenzoateEX Objective\n=========\n1.0 BIOMASS = 0.00900004718130796\n\nUptake\n------\n         Metabolite             Reaction      Flux  C-Number C-Flux\n         Cadmium[e]            CadmiumEX 4.344E-07         0  0.00%\n  Calcium_cation[e]            CalciumEX 0.0002409         0  0.00%\n    Chloride_ion[e]           ChlorideEX  0.002141         0  0.

L_GlutamateEX GlycogenEX
L_GlutamateEX L_ArginineEX
TimeoutExpired 20
stdout: b'Scaling...\n A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00\nProblem data seem to be well scaled\nL_GlutamateEX L_ArginineEX Objective\n=========\n1.0 BIOMASS = 0.009000033950732548\n\nUptake\n------\n         Metabolite             Reaction      Flux  C-Number C-Flux\n         Cadmium[e]            CadmiumEX 4.344E-07         0  0.00%\n  Calcium_cation[e]            CalciumEX 0.0002409         0  0.00%\n    Chloride_ion[e]           ChlorideEX  0.002141         0  0.00%\n      Cobalt_ion[e]             CobaltEX 1.397E-05         0  0.00%\n          Copper[e]             CopperEX  7.86E-07         0  0.00%\n  Orthophosphate[e] FAKEOrthophosphateEX   0.00327         0  0.00%\n            HCO3[e]            HCO3EXcar     5.367         1 19.60%\n             Fe2[e]               IronEX 2.909E-05         0  0.00%\n     L_Glutamate[e]        L_GlutamateEX     4.402         5 80.40%\n        

L_ArginineEX SAdenosyl4methylthio2oxobutanoateEX
L_ArginineEX 4_AminobenzoateEX
TimeoutExpired 20
stdout: b'Scaling...\n A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00\nProblem data seem to be well scaled\nL_ArginineEX 4_AminobenzoateEX Objective\n=========\n1.0 BIOMASS = 0.009000131426240099\n\nUptake\n------\n         Metabolite             Reaction      Flux  C-Number C-Flux\n         Cadmium[e]            CadmiumEX 4.344E-07         0  0.00%\n  Calcium_cation[e]            CalciumEX 0.0002409         0  0.00%\n    Chloride_ion[e]           ChlorideEX  0.002141         0  0.00%\n      Cobalt_ion[e]             CobaltEX 1.397E-05         0  0.00%\n          Copper[e]             CopperEX  7.86E-07         0  0.00%\n  Orthophosphate[e] FAKEOrthophosphateEX   0.00327         0  0.00%\n            HCO3[e]            HCO3EXcar     4.755         1 75.21%\n             Fe2[e]               IronEX 2.909E-05         0  0.00%\n      L_Arginine[e]         L_ArginineEX    0

TimeoutExpired 20
stdout: b'Scaling...\n A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00\nProblem data seem to be well scaled\nL_ValineEX L_GlutamineEX Objective\n=========\n1.0 BIOMASS = 0.00900007964341205\n\nUptake\n------\n         Metabolite             Reaction      Flux  C-Number C-Flux\n         Cadmium[e]            CadmiumEX 4.344E-07         0  0.00%\n  Calcium_cation[e]            CalciumEX 0.0002409         0  0.00%\n    Chloride_ion[e]           ChlorideEX  0.002141         0  0.00%\n      Cobalt_ion[e]             CobaltEX 1.397E-05         0  0.00%\n          Copper[e]             CopperEX  7.86E-07         0  0.00%\n  Orthophosphate[e] FAKEOrthophosphateEX   0.00327         0  0.00%\n            HCO3[e]            HCO3EXcar     4.759         1 50.41%\n             Fe2[e]               IronEX 2.909E-05         0  0.00%\n        L_Valine[e]           L_ValineEX    0.9364         5 49.59%\n          Photon[e]              LightEX     45.17         0  0

L_ValineEX LPSEX
L_ValineEX AcetateEX
TimeoutExpired 20
stdout: b'Scaling...\n A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00\nProblem data seem to be well scaled\nL_ValineEX AcetateEX Objective\n=========\n1.0 BIOMASS = 0.009000030771152445\n\nUptake\n------\n         Metabolite             Reaction      Flux  C-Number C-Flux\n         Cadmium[e]            CadmiumEX 4.344E-07         0  0.00%\n  Calcium_cation[e]            CalciumEX 0.0002409         0  0.00%\n    Chloride_ion[e]           ChlorideEX  0.002141         0  0.00%\n      Cobalt_ion[e]             CobaltEX 1.397E-05         0  0.00%\n          Copper[e]             CopperEX  7.86E-07         0  0.00%\n  Orthophosphate[e] FAKEOrthophosphateEX   0.00327         0  0.00%\n            HCO3[e]            HCO3EXcar     4.759         1 92.41%\n             Fe2[e]               IronEX 2.909E-05         0  0.00%\n        L_Valine[e]           L_ValineEX   0.07817         5  7.59%\n          Photon[e]         

In [78]:
len(knockout_success)

n_exchange_df.loc[n_exchange_df.knockout_success2 == 'timeout', 'knockout_success3'] = knockout_success

/home/oweissberg/work/tools/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)


In [79]:
n_exchange_df.loc[
    n_exchange_df.knockout_success2 != 'timeout', 'knockout_success3'
] = n_exchange_df.loc[
    n_exchange_df.knockout_success2 != 'timeout', 'knockout_success2'
]

In [80]:
n_exchange_df.knockout_success3.value_counts()

success    677
error      180
timeout     24
Name: knockout_success3, dtype: int64

In [81]:
tdf = n_exchange_df.loc[n_exchange_df.knockout_success3 != 'success']
dpath = 'gene_knockout_secreted_18082022'
knockout_success = [
    _gene_deletion_feeding_with_timeout(dpath, u, s, m, timeout=20)
    for (u, s, m) in zip(tdf['uptake'], tdf['secretion'], tdf['maximum'])
]


GuanosineEX L_AsparagineEX
CalledProcessError 1
stdout: Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled

stderr: Traceback (most recent call last):
  File "/fast_data/Osnat/CC1A3/COMETS/Prochlorococcus_Model/N_Starvation/./prochlorococcus_feed_knockout.py", line 123, in <module>
    resdf = _gene_deletion_feeding(model, args.uptake, args.secretion, args.bound)
  File "/fast_data/Osnat/CC1A3/COMETS/Prochlorococcus_Model/N_Starvation/./prochlorococcus_feed_knockout.py", line 93, in _gene_deletion_feeding
    print(uptake1, sec1, model.summary())
  File "/home/oweissberg/work/tools/anaconda3/lib/python3.9/site-packages/cobra/core/model.py", line 1209, in summary
    return ModelSummary(model=self, solution=solution, fva=fva)
  File "/home/oweissberg/work/tools/anaconda3/lib/python3.9/site-packages/cobra/summary/model_summary.py", line 85, in __init__
    self._generate(model, solution, fva)
  File "/home/oweissberg/work/t

CalledProcessError 1
stdout: Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled

stderr: Traceback (most recent call last):
  File "/fast_data/Osnat/CC1A3/COMETS/Prochlorococcus_Model/N_Starvation/./prochlorococcus_feed_knockout.py", line 123, in <module>
    resdf = _gene_deletion_feeding(model, args.uptake, args.secretion, args.bound)
  File "/fast_data/Osnat/CC1A3/COMETS/Prochlorococcus_Model/N_Starvation/./prochlorococcus_feed_knockout.py", line 93, in _gene_deletion_feeding
    print(uptake1, sec1, model.summary())
  File "/home/oweissberg/work/tools/anaconda3/lib/python3.9/site-packages/cobra/core/model.py", line 1209, in summary
    return ModelSummary(model=self, solution=solution, fva=fva)
  File "/home/oweissberg/work/tools/anaconda3/lib/python3.9/site-packages/cobra/summary/model_summary.py", line 85, in __init__
    self._generate(model, solution, fva)
  File "/home/oweissberg/work/tools/anaconda3/lib/python3.

CalledProcessError 1
stdout: Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled

stderr: Traceback (most recent call last):
  File "/fast_data/Osnat/CC1A3/COMETS/Prochlorococcus_Model/N_Starvation/./prochlorococcus_feed_knockout.py", line 123, in <module>
    resdf = _gene_deletion_feeding(model, args.uptake, args.secretion, args.bound)
  File "/fast_data/Osnat/CC1A3/COMETS/Prochlorococcus_Model/N_Starvation/./prochlorococcus_feed_knockout.py", line 93, in _gene_deletion_feeding
    print(uptake1, sec1, model.summary())
  File "/home/oweissberg/work/tools/anaconda3/lib/python3.9/site-packages/cobra/core/model.py", line 1209, in summary
    return ModelSummary(model=self, solution=solution, fva=fva)
  File "/home/oweissberg/work/tools/anaconda3/lib/python3.9/site-packages/cobra/summary/model_summary.py", line 85, in __init__
    self._generate(model, solution, fva)
  File "/home/oweissberg/work/tools/anaconda3/lib/python3.

CalledProcessError 1
stdout: Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled

stderr: Traceback (most recent call last):
  File "/fast_data/Osnat/CC1A3/COMETS/Prochlorococcus_Model/N_Starvation/./prochlorococcus_feed_knockout.py", line 123, in <module>
    resdf = _gene_deletion_feeding(model, args.uptake, args.secretion, args.bound)
  File "/fast_data/Osnat/CC1A3/COMETS/Prochlorococcus_Model/N_Starvation/./prochlorococcus_feed_knockout.py", line 93, in _gene_deletion_feeding
    print(uptake1, sec1, model.summary())
  File "/home/oweissberg/work/tools/anaconda3/lib/python3.9/site-packages/cobra/core/model.py", line 1209, in summary
    return ModelSummary(model=self, solution=solution, fva=fva)
  File "/home/oweissberg/work/tools/anaconda3/lib/python3.9/site-packages/cobra/summary/model_summary.py", line 85, in __init__
    self._generate(model, solution, fva)
  File "/home/oweissberg/work/tools/anaconda3/lib/python3.

CalledProcessError 1
stdout: Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled

stderr: Traceback (most recent call last):
  File "/fast_data/Osnat/CC1A3/COMETS/Prochlorococcus_Model/N_Starvation/./prochlorococcus_feed_knockout.py", line 123, in <module>
    resdf = _gene_deletion_feeding(model, args.uptake, args.secretion, args.bound)
  File "/fast_data/Osnat/CC1A3/COMETS/Prochlorococcus_Model/N_Starvation/./prochlorococcus_feed_knockout.py", line 93, in _gene_deletion_feeding
    print(uptake1, sec1, model.summary())
  File "/home/oweissberg/work/tools/anaconda3/lib/python3.9/site-packages/cobra/core/model.py", line 1209, in summary
    return ModelSummary(model=self, solution=solution, fva=fva)
  File "/home/oweissberg/work/tools/anaconda3/lib/python3.9/site-packages/cobra/summary/model_summary.py", line 85, in __init__
    self._generate(model, solution, fva)
  File "/home/oweissberg/work/tools/anaconda3/lib/python3.

...

CalledProcessError 1
stdout: Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled

stderr: Traceback (most recent call last):
  File "/fast_data/Osnat/CC1A3/COMETS/Prochlorococcus_Model/N_Starvation/./prochlorococcus_feed_knockout.py", line 123, in <module>
    resdf = _gene_deletion_feeding(model, args.uptake, args.secretion, args.bound)
  File "/fast_data/Osnat/CC1A3/COMETS/Prochlorococcus_Model/N_Starvation/./prochlorococcus_feed_knockout.py", line 93, in _gene_deletion_feeding
    print(uptake1, sec1, model.summary())
  File "/home/oweissberg/work/tools/anaconda3/lib/python3.9/site-packages/cobra/core/model.py", line 1209, in summary
    return ModelSummary(model=self, solution=solution, fva=fva)
  File "/home/oweissberg/work/tools/anaconda3/lib/python3.9/site-packages/cobra/summary/model_summary.py", line 85, in __init__
    self._generate(model, solution, fva)
  File "/home/oweissberg/work/tools/anaconda3/lib/python3.

CalledProcessError 1
stdout: Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled

stderr: Traceback (most recent call last):
  File "/fast_data/Osnat/CC1A3/COMETS/Prochlorococcus_Model/N_Starvation/./prochlorococcus_feed_knockout.py", line 123, in <module>
    resdf = _gene_deletion_feeding(model, args.uptake, args.secretion, args.bound)
  File "/fast_data/Osnat/CC1A3/COMETS/Prochlorococcus_Model/N_Starvation/./prochlorococcus_feed_knockout.py", line 93, in _gene_deletion_feeding
    print(uptake1, sec1, model.summary())
  File "/home/oweissberg/work/tools/anaconda3/lib/python3.9/site-packages/cobra/core/model.py", line 1209, in summary
    return ModelSummary(model=self, solution=solution, fva=fva)
  File "/home/oweissberg/work/tools/anaconda3/lib/python3.9/site-packages/cobra/summary/model_summary.py", line 85, in __init__
    self._generate(model, solution, fva)
  File "/home/oweissberg/work/tools/anaconda3/lib/python3.

CalledProcessError 1
stdout: Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled

stderr: Traceback (most recent call last):
  File "/fast_data/Osnat/CC1A3/COMETS/Prochlorococcus_Model/N_Starvation/./prochlorococcus_feed_knockout.py", line 123, in <module>
    resdf = _gene_deletion_feeding(model, args.uptake, args.secretion, args.bound)
  File "/fast_data/Osnat/CC1A3/COMETS/Prochlorococcus_Model/N_Starvation/./prochlorococcus_feed_knockout.py", line 93, in _gene_deletion_feeding
    print(uptake1, sec1, model.summary())
  File "/home/oweissberg/work/tools/anaconda3/lib/python3.9/site-packages/cobra/core/model.py", line 1209, in summary
    return ModelSummary(model=self, solution=solution, fva=fva)
  File "/home/oweissberg/work/tools/anaconda3/lib/python3.9/site-packages/cobra/summary/model_summary.py", line 85, in __init__
    self._generate(model, solution, fva)
  File "/home/oweissberg/work/tools/anaconda3/lib/python3.

TimeoutExpired 20
stdout: b'Scaling...\n A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00\nProblem data seem to be well scaled\n'
stderr: None
CyanateEX 4_Methyl_2_oxopentanoateEX
CalledProcessError 1
stdout: Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled

stderr: Traceback (most recent call last):
  File "/fast_data/Osnat/CC1A3/COMETS/Prochlorococcus_Model/N_Starvation/./prochlorococcus_feed_knockout.py", line 123, in <module>
    resdf = _gene_deletion_feeding(model, args.uptake, args.secretion, args.bound)
  File "/fast_data/Osnat/CC1A3/COMETS/Prochlorococcus_Model/N_Starvation/./prochlorococcus_feed_knockout.py", line 93, in _gene_deletion_feeding
    print(uptake1, sec1, model.summary())
  File "/home/oweissberg/work/tools/anaconda3/lib/python3.9/site-packages/cobra/core/model.py", line 1209, in summary
    return ModelSummary(model=self, solution=solution, fva=fva)
  File "/home/oweissberg/wor

CalledProcessError 1
stdout: Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled

stderr: Traceback (most recent call last):
  File "/fast_data/Osnat/CC1A3/COMETS/Prochlorococcus_Model/N_Starvation/./prochlorococcus_feed_knockout.py", line 123, in <module>
    resdf = _gene_deletion_feeding(model, args.uptake, args.secretion, args.bound)
  File "/fast_data/Osnat/CC1A3/COMETS/Prochlorococcus_Model/N_Starvation/./prochlorococcus_feed_knockout.py", line 93, in _gene_deletion_feeding
    print(uptake1, sec1, model.summary())
  File "/home/oweissberg/work/tools/anaconda3/lib/python3.9/site-packages/cobra/core/model.py", line 1209, in summary
    return ModelSummary(model=self, solution=solution, fva=fva)
  File "/home/oweissberg/work/tools/anaconda3/lib/python3.9/site-packages/cobra/summary/model_summary.py", line 85, in __init__
    self._generate(model, solution, fva)
  File "/home/oweissberg/work/tools/anaconda3/lib/python3.

In [82]:
len(knockout_success)


204

In [86]:

n_exchange_df.loc[n_exchange_df.knockout_success3 != 'success', 'knockout_success4'] = knockout_success

In [87]:
n_exchange_df.loc[
    n_exchange_df.knockout_success3 == 'success', 'knockout_success4'
] = n_exchange_df.loc[
    n_exchange_df.knockout_success3 == 'success', 'knockout_success3'
]

In [88]:
n_exchange_df.knockout_success4.value_counts()

success    692
error      180
timeout      9
Name: knockout_success4, dtype: int64

In [89]:
dpath = 'gene_knockout_secreted_18082022'
os.listdir(dpath)

['secretion_knockout_1gene_L_ValineEX_AmmoniaEX.csv',
 'secretion_knockout_1gene_GlutathioneEX_PantothenateEX.csv',
 'secretion_knockout_1gene_L_CystineEX_SpermidineEX.csv',
 'secretion_knockout_1gene_L_GlutamateEX_4_Methyl_2_oxopentanoateEX.csv',
 'secretion_knockout_1gene_L_AlanineEX_PantothenateEX.csv',
 'secretion_knockout_1gene_GuanosineEX_SpermineEX.csv',
 'secretion_knockout_1gene_L_ArginineEX_H2O2EX.csv',
 'secretion_knockout_1gene_L_LeucineEX_L_TyrosineEX.csv',
 'secretion_knockout_1gene_L_SerineEX_L_ThreonineEX.csv',
 'secretion_knockout_1gene_L_AspartateEX_L_SerineEX.csv',
 'secretion_knockout_1gene_L_LeucineEX_L_ValineEX.csv',
 'secretion_knockout_1gene_GuanosineEX_SpermidineEX.csv',
 'secretion_knockout_1gene_UreaEX_5_MethylthioadenosineEX.csv',
 'secretion_knockout_1gene_L_ValineEX_5_MethylthioadenosineEX.csv',
 'secretion_knockout_1gene_L_LeucineEX_L_AlanineEX.csv',
 'secretion_knockout_1gene_L_AspartateEX_L_ValineEX.csv',
 'secretion_knockout_1gene_L_AspartateEX_Putresc

In [91]:
knockout_df = pd.concat([pd.read_csv(os.path.join(dpath, i)) for i in os.listdir(dpath)])

In [93]:
knockout_df.groupby(['ids','uptake'])['secretion'].count()

ids          uptake       
{'PMM0003'}  AmmoniaEX        10
             CyanateEX         9
             GlutathioneEX    12
             GlycineEX         2
             GuanineEX         7
                              ..
{'PMM1716'}  L_GlutamineEX     2
             L_LeucineEX       1
             L_SerineEX        2
             L_ValineEX        2
             UreaEX            1
Name: secretion, Length: 4284, dtype: int64

In [96]:
knockout_df.groupby(['ids'])['secretion'].count().nlargest(30)

ids
{'PMM1438'}    629
{'PMM1439'}    629
{'PMM1450'}    629
{'PMM1451'}    629
{'PMM1452'}    629
{'PMM1453'}    629
{'PMM1454'}    629
{'PMM1455'}    629
{'PMM1456'}    629
{'PMM1489'}    625
{'PMM0023'}    602
{'PMM0195'}    602
{'PMM1235'}    596
{'PMM1236'}    596
{'PMM1610'}    596
{'PMM0766'}    595
{'PMM0213'}    582
{'PMM0781'}    580
{'PMM0829'}    580
{'PMM0550'}    547
{'PMM0551'}    547
{'PMM0785'}    547
{'PMM0552'}    545
{'PMM0553'}    545
{'PMM0554'}    545
{'PMM0555'}    545
{'PMM0325'}    500
{'PMM0326'}    500
{'PMM0329'}    500
{'PMM0461'}    500
Name: secretion, dtype: int64

In [100]:
knockout_df.pivot_table(index=['ids'], columns=['secretion'], aggfunc='count', values='uptake')

secretion,4_AminobenzoateEX,4_Methyl_2_oxopentanoateEX,5_MethylthioadenosineEX,AcetateEX,AdenineEX,AmmoniaEX,B12EX,BiotinEX,CitrateEX,DethiobiotinEX,...,SAdenosyl4methylthio2oxobutanoateEX,SMethyl5thioDriboseEX,S_MalateEX,SpermidineEX,SpermineEX,SuccinateEX,ThiosulfateEX,ThymidineEX,UreaEX,XanthosineEX
ids,,,,,,,,,,,,,,,,,,,,,
{'PMM0003'},NaN,NaN,13.0,NaN,12.0,NaN,13.0,11.0,NaN,12.0,...,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.0
{'PMM0004'},NaN,NaN,13.0,NaN,12.0,NaN,13.0,11.0,NaN,12.0,...,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.0
{'PMM0012'},NaN,NaN,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,9.0,NaN,10.0,8.0,NaN,NaN,NaN,11.0,NaN
{'PMM0023'},16.0,9.0,12.0,10.0,10.0,5.0,12.0,10.0,11.0,11.0,...,11.0,8.0,15.0,10.0,7.0,12.0,3.0,14.0,11.0,14.0
{'PMM0027'},NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.0,NaN,12.0,...,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
{'PMM1700'},NaN,NaN,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,7.0,NaN,7.0,6.0,1.0,NaN,NaN,8.0,NaN
{'PMM1702'},NaN,NaN,NaN,NaN,NaN,NaN,8.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
{'PMM1705'},16.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [101]:
knockout_df.pivot_table(index=['ids'], columns=['uptake'], aggfunc='count', values='secretion')

uptake,AmmoniaEX,CyanateEX,GlutathioneEX,GlycineEX,GuanineEX,GuanosineEX,L_AlanineEX,L_ArginineEX,L_AspartateEX,L_CystineEX,L_GlutamateEX,L_GlutamineEX,L_LeucineEX,L_SerineEX,L_ValineEX,UreaEX
ids,,,,,,,,,,,,,,,,
{'PMM0003'},10.0,9.0,12.0,2.0,7.0,7.0,9.0,9.0,12.0,6.0,7.0,11.0,9.0,6.0,11.0,10.0
{'PMM0004'},10.0,9.0,12.0,2.0,7.0,7.0,9.0,9.0,12.0,6.0,7.0,11.0,9.0,6.0,11.0,10.0
{'PMM0012'},6.0,4.0,7.0,1.0,7.0,7.0,6.0,NaN,5.0,4.0,5.0,7.0,5.0,NaN,7.0,3.0
{'PMM0023'},37.0,38.0,44.0,16.0,55.0,27.0,42.0,33.0,42.0,35.0,41.0,48.0,31.0,21.0,52.0,40.0
{'PMM0027'},5.0,5.0,5.0,NaN,5.0,5.0,2.0,4.0,5.0,NaN,2.0,5.0,5.0,NaN,5.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
{'PMM1700'},8.0,8.0,NaN,1.0,11.0,12.0,9.0,2.0,8.0,5.0,NaN,NaN,7.0,NaN,11.0,6.0
{'PMM1702'},4.0,1.0,3.0,NaN,3.0,14.0,2.0,1.0,9.0,2.0,2.0,3.0,3.0,7.0,3.0,1.0
{'PMM1705'},3.0,3.0,4.0,2.0,4.0,4.0,4.0,3.0,4.0,3.0,3.0,4.0,3.0,4.0,4.0,3.0


In [99]:
n_exchange_df.secretion.nunique()

56